In [ ]:

import rasterio
import numpy as np
import matplotlib.pyplot as plt 
from datetime import date
import openeo

In [ ]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu").authenticate_oidc()